In [ ]:
import pandas as pd


#파일 넣기기
input_file = "서울대기오염_2019"

#1. 파일 확인: 
df = pd.read_excel(input_file)

print(df.info())
print(df.isnull().sum())

df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9491 entries, 0 to 9490
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   날짜               9491 non-null   object 
 1   측정소명             9491 non-null   object 
 2   미세먼지             9278 non-null   float64
 3   초미세먼지            9288 non-null   float64
 4   오존               9294 non-null   float64
 5   이산화질소
NO2 (ppm)  9285 non-null   float64
 6   일산화탄소
CO (ppm)   9234 non-null   float64
 7   아황산가스
SO2(ppm)   9298 non-null   float64
dtypes: float64(6), object(2)
memory usage: 593.3+ KB
None
              미세먼지        초미세먼지           오존  이산화질소\nNO2 (ppm)  \
count  9278.000000  9288.000000  9294.000000       9285.000000   
mean     41.748006    24.921619     0.024771          0.028068   
std      26.742015    18.410941     0.013191          0.012733   
min       3.000000     1.000000     0.001000          0.001000   
25%      24.000000    14.000000     0.014000 

In [7]:
df.isnull().sum()

날짜                    0
측정소명                  0
미세먼지                213
초미세먼지               203
오존                  197
이산화질소\nNO2 (ppm)    206
일산화탄소\nCO (ppm)     257
아황산가스\nSO2(ppm)     193
dtype: int64